In [1]:
from pathlib import Path
import glob
import numpy as np
import pandas as pd
import json
import pyarrow.parquet as pq
import os
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime
from datetime import timedelta

1. Parquet-Dateien in pandas Dataframe laden
- für große Datenmengen konzipiert/ Speicherung der Daten in spaltenorientiertem Format
 - Für jeden Tag sind mehrere hundert bis bis mehrere tausend Parquet vorhanden (je nach Laufdauer und Datenproduktion der Maschine)
 - Jede Parquet-Datei enthält ca. eine halbe Million Zeilen
 - Bibliotheken: pandas, numpy


In [5]:
#change date accordingly - specify source and destination folders
date = "28.04"
orig_folder = 'C:/Users/signa/_Relkat/data/data_raw/'
dest_folder = 'C:/Users/signa/_Relkat/data/data_csv/'+date+"/"
filename = "MD_"+date
print(orig_folder)

C:/Users/signa/_Relkat/data/data_raw/


In [10]:
example_file = pd.read_parquet(orig_folder+filename+'/'+'0006-utc-1619568410315.parquet')
print(example_file.shape)
example_file.head(3)

(409959, 9)


,uuid,sequence_number,sys_time,plc_time,value_integer,value_string,value_double,value_bool,value_bytes
0,0073976b-2f3d-49e6-b966-70051c19239f,280094109,2021-04-28 00:06:50.315506+00:00,2021-04-28 00:06:57.397000+00:00,NaN,None,0.0,None,None
1,15132e77-9aa6-4248-b88e-efc39d3dd1df,280094109,2021-04-28 00:06:50.315506+00:00,2021-04-28 00:06:57.397000+00:00,NaN,None,0.0,None,None
2,1780640a-42ac-4419-b830-76726b513b7d,280094109,2021-04-28 00:06:50.315506+00:00,2021-04-28 00:06:57.397000+00:00,NaN,None,0.0,None,None


In [16]:
parquet_files = [f for f in os.listdir(os.path.join(orig_folder,filename))]
n_files = len(parquet_files)
print("for ",filename,"total number of files available: "+ str(n_files))

for  MD_28.04 total number of files available: 2025


2. Einzelne Parquet-Dateien nach gesuchten Daten filtern und zu einer Datei zusammenfügen
- herausfiltern der Maschinendaten für die drei Stellgrößen bzw. andere Prozessdaten durch einen regulären Ausdruck
- portionsweises konkatenieren (jeweils 10 Dateien, damit das RAM nicht überläuft)
- Erzeugen einer eigenen Spalte ("label") zur Beschreibung des jeweiligen Datenpunktes aus den Metadata(im JSON-Format) über Konkatenierung mit einem Identifikator
- Gefilterte Maschinendaten haben im Beispielfall nur noch einen Umfang von ca. 26000 Zeilen
- Anordnung der Daten untereinander nach Systemzeit 
- Prozessdaten werden normalerweise alle 10 ms gemessen, wurden hier aber auf 1 Hz interpoliert.
- Bibliotheken: re (für reguläre Ausdrücke, json, pandas)

In [17]:
def concat_single_files(files):
    frames = []
    for idf_file in range(len(files)):
        file_chunk = pd.read_parquet(orig_folder+filename+'/'+files[idf_file])
        df_mapped = file_chunk.merge(df_mapping, on='uuid', how='inner')
        mask = (df_mapped['label'].str.contains(MD_filter, na=False)) 
        df_takt = df_mapped.loc[mask]
        frames.append(df_takt)
    df_concat = pd.concat(frames)
    return df_concat

In [18]:
import re
start_file = 0
end_file = n_files
    
MD_filter = re.compile("(^(?!Sollwert freigeben).*)(Taktzeit 1 Start|Hydraulik: externe|DB_KÜHLMITTEL_HOCHDRUCK.Sollwert|(DB_KÜHLMITTEL.Sollwert ))([a-z])*",flags=re.IGNORECASE)
    
#MD_filter = re.compile("(Taktzeit 1 Start|Drehzahlvorgabe|Drezahl)([a-z])*",flags=re.IGNORECASE)
#MD_filter = re.compile("([a-z])*(Durchfluss)([a-z])*",flags=re.IGNORECASE)

parquet_files = [f for f in os.listdir(os.path.join(orig_folder,filename))][start_file:end_file]
print("Total number of files to process: "+ str(len(parquet_files)))

table = pq.read_table(os.path.join(orig_folder,filename+'/'+parquet_files[0]))
metadata_b = table.schema.metadata.get(b'json_meta_data')
metadata = json.loads(metadata_b)
datapoints = metadata['data_points']
mapping = [ { 'uuid': meta['uuid'], 'identifier': meta['identifier'],
            'label': meta['label']  } for meta in datapoints]
df_mapping = pd.DataFrame(mapping)
chunks = []
n_chunks = 10
for i in range(len(parquet_files)//10):    
    df_concat = concat_single_files(parquet_files[i*n_chunks: (i+1)*n_chunks])
    chunks.append(df_concat)
    if((i+1)*n_chunks%100==0):
        print(str(round(((i+1)*n_chunks/len(parquet_files)*100),2))+" % done")
df_MD = pd.concat(chunks)
print(df_MD.shape)

Total number of files to process: 2025


C:\Users\signa\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


4.94 % done
9.88 % done
14.81 % done
19.75 % done
24.69 % done
29.63 % done
34.57 % done
39.51 % done
44.44 % done
49.38 % done
54.32 % done
59.26 % done
64.2 % done
69.14 % done
74.07 % done
79.01 % done
83.95 % done
88.89 % done
93.83 % done
98.77 % done
(26590, 11)


In [110]:
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_colwidth', 100)
df_MD.reset_index(drop=True,inplace=True)
df_MD.head(200)

,Unnamed: 0,uuid,sequence_number,sys_time,plc_time,value_integer,value_string,value_double,value_bool,value_bytes,identifier,label
0,376925,b4391adb-f3f3-440a-b952-0f9aad38af6e,3,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,E 48.2,Kühlschmierstoff (Hochdruck): Durchfluss ok (i_225_92 )
1,376942,8ed35711-277f-4e6d-a897-79ac7d4cddc0,20,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,E 35.4,Absaugung 1: Durchfluss ok (i_250_07 )
2,376952,e9b8201d-406b-4a12-8745-22d5bca23395,30,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,M 2001.3,Kühlaggregat: Durchfluss ok (Maschine) (mi_130_11 )
3,376954,be361977-44cd-4006-a70e-84e50a9d6686,32,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,M 2001.1,Kühlaggregat: Durchfluss ok (Schaltschrank) (mi_130_09 )
4,377003,7d89647f-cf77-4839-9dbf-0f37e66e4aa9,81,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,M 1990.6,Kühlwasser: Durchfluss Spindelstock (mi_130_92 )
5,377004,489225af-a7ba-41ea-b5aa-fafb6eb2eec9,82,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,M 1990.5,Kühlwasser: Durchfluss Revolver 1 (mi_130_91 )
6,377005,7d5a84e1-2e14-4172-8bd5-e93403892189,83,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,M 1990.4,Kühlwasser: Durchfluss X1-Achse (mi_130_90 )
7,377301,9f5f7900-2907-4a87-b0c2-5ba7d4869e8b,146,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,E 74.0,Kühlaggregat: Durchfluss ok (Revolver 1) (i_130_28 )
8,377304,c1c1b11e-b81d-4751-adf5-e3b89c6ff35d,149,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,E 74.1,Kühlaggregat: Durchfluss ok (Spindelkasten 1) (i_130_18 )
9,377320,ee5db2b0-028a-45c3-a981-6f40df7d3c4a,165,2021-02-18 04:21:00.604200694,2021-02-18 03:21:00.482500+00:00,NaN,NaN,NaN,False,NaN,E 74.2,Kühlaggregat: Durchfluss ok (Revolver 2) (i_130_29 )


In [96]:
# Remove duplicate entries
# Normally there are no duplicates
print(df_MD.shape)
df_MD.drop_duplicates(inplace=True)
print(df_MD.shape)

(1079679, 11)
(1079679, 11)


In [ ]:
# AUF DIE ZEIT IN DEN MASCHINENDATEN MUESSEN 1H BZW 2H GEGENÜBER DER ORTSZEIT AUFADDIERT WERDEN!!!

In [109]:
df_MD['sys_time'] = df_MD['sys_time'].apply(lambda x: pd.to_datetime(x))
df_MD['sys_time'] = df_MD['sys_time'].apply(lambda x: x.tz_localize(None))
if ((date=='16.02')|(date=='18.02')):
        df_MD['sys_time'] = df_MD['sys_time'].apply(lambda x: x + timedelta(hours = 1))
if ((date=='26.04')|(date=='28.04')):
        df_MD['sys_time'] = df_MD['sys_time'].apply(lambda x: x + timedelta(hours = 2))

In [111]:
# specify filename for specific data
filename_dz = "MD_Durchfluss_"+date
df_MD.to_csv(os.path.join(dest_folder,filename_dz)+".csv")
print(os.path.join(dest_folder,filename_dz)+".csv")

C:/Users/signa/_Relkat/data/data_csv/28.04/MD_Durchfluss_18.02.csv


In [33]:
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_colwidth', 100)
print(len(df_mapping))
df_mapping['label']

606


0                                                               Kühlschmierstoff Sammelstörung (i_230_95 )
1                            Kühlschmierstoff Temperarutstabisilierung: Motorschutzschalter ok (i_230_90 )
2                                                  Kühlschmierstoff (Hochdruck): Durchfluss ok (i_225_92 )
3                                                  Kühlschmierstoff (Hochdruck): Motor Störung (i_225_91 )
4                                           Kühlschmierstoff (Hochdruck): Motor betriebsbereit (i_225_90 )
5                                         Kühlschmierstoff (Hochdruck): Motorschutzschalter ok (i_225_01 )
6                                                              Kühlschmierstoff: Motor Störung (i_220_91 )
7                                                       Kühlschmierstoff: Motor betriebsbereit (i_220_90 )
8                                       Spannstock 1 (Kanal 1): Kühlschmierstoff ein (Spülen) (mo_642_05 )
9                                    